# Leonardo's Mechanical Odometer: Measuring the World

> *"Mechanics is the paradise of the mathematical sciences, because by means of it one comes to the fruits of mathematics."*  
> — Leonardo da Vinci

## Introduction
Leonardo designed a cart that would drop a pebble into a bucket for every revolution of a wheel, allowing for precise measurement of distances. This **odometer** was intended for mapping and surveying.

In this notebook, we simulate the measurement process and analyze sources of error:
1.  **Gear Ratios**: How wheel turns translate to pebble drops.
2.  **Systematic Error**: Calibration issues (e.g., wheel diameter).
3.  **Random Error**: Wheel slip on different terrains.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Measurement Principle
The distance $D$ is calculated by counting pebbles $N$:
$$ D = N \cdot C_{wheel} \cdot R_{gear} $$
Where $C_{wheel}$ is the wheel circumference and $R_{gear}$ is the gear ratio (drops per wheel turn).

### 2. Error Sources
- **Slip**: The wheel slides instead of rolling (random error).
- **Wear**: The wheel diameter decreases over time (systematic error).
- **Terrain**: Rough ground causes bouncing and slip.


In [ ]:
def simulate_measurement(distance_m, terrain_type, wheel_diam_m):
    # Parameters
    circumference = np.pi * wheel_diam_m
    gear_ratio = 10.0 # 1 drop per 10 turns
    distance_per_drop = circumference * gear_ratio
    
    # Terrain Factors (Slip %)
    terrain_slip = {
        'Paved': 0.01,
        'Dirt': 0.05,
        'Grass': 0.10,
        'Mud': 0.20
    }
    slip_factor = terrain_slip[terrain_type]
    
    # Simulation
    # Actual distance traveled vs. Wheel distance measured
    # Wheel distance = Actual / (1 - slip)
    # But we measure drops based on wheel distance
    
    # Let's simulate step by step for random slip
    steps = int(distance_m / 1.0) # 1m steps
    measured_dist = 0.0
    
    actual_path = []
    measured_path = []
    
    for i in range(steps):
        step_dist = 1.0
        # Random slip for this step
        current_slip = np.random.normal(slip_factor, slip_factor * 0.5)
        current_slip = np.clip(current_slip, 0, 0.5)
        
        # Wheel turns MORE than actual distance due to slip
        wheel_dist_step = step_dist / (1.0 - current_slip)
        measured_dist += wheel_dist_step
        
        actual_path.append((i+1))
        measured_path.append(measured_dist)
        
    # Calculate Drops
    drops = int(measured_dist / distance_per_drop)
    calculated_distance = drops * distance_per_drop
    
    error = calculated_distance - distance_m
    error_percent = (error / distance_m) * 100
    
    return actual_path, measured_path, calculated_distance, error_percent, drops

def plot_odometer(distance, terrain, diameter):
    actual, measured, calc_dist, error, drops = simulate_measurement(distance, terrain, diameter)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(actual, actual, 'k--', label='True Distance')
    ax.plot(actual, measured, 'r-', label='Measured (Wheel)')
    
    ax.set_title(f'Odometer Accuracy on {terrain}')
    ax.set_xlabel('Actual Distance (m)')
    ax.set_ylabel('Measured Distance (m)')
    ax.legend()
    
    plt.show()
    
    print(f"Pebbles Dropped: {drops}")
    print(f"Calculated Distance: {calc_dist:.1f} m")
    print(f"Actual Distance: {distance:.1f} m")
    print(f"Error: {error:.1f} m ({error:.1f}%)")
    print("Note: Positive error means the odometer overestimates distance due to wheel slip.")

interact(plot_odometer, 
         distance=widgets.IntSlider(min=100, max=2000, step=100, value=1000, description='Distance (m)'),
         terrain=widgets.Dropdown(options=['Paved', 'Dirt', 'Grass', 'Mud'], value='Dirt', description='Terrain'),
         diameter=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='Wheel Diam (m)'));

## Conclusion

The simulation demonstrates that **wheel slip** is a significant source of error, especially on loose terrain. The odometer consistently *overestimates* distance because the wheel spins without moving forward effectively.

Leonardo likely understood this and would have calibrated his device for specific road conditions, or used it primarily on the paved Roman roads that still existed in Italy.